In [1]:
import tensorflow as tf

In [4]:
num_dims = 6

votes_t_shape = [3, 0, 1, 2]
for i in range(num_dims - 4):
    votes_t_shape += [i + 4]  # CONV: votes_t_shape - [3, 0, 1, 2, 4, 5]
    
r_t_shape = [1, 2, 3, 0]
for i in range(num_dims - 4):
    r_t_shape += [i + 4]  # CONV: votes_t_shape - [1, 2, 3, 0, 4, 5]

# 第三维度提到最前
votes_trans = tf.transpose(votes, votes_t_shape)

NameError: name 'votes' is not defined

In [42]:
import numpy as np
import tensorflow as tf

# data
x_train = np.random.random([100, 3])
y_train = np.rint(np.random.random([100, 2]))
 
epochs = 30
display_step = 2


tf.reset_default_graph()
train_graph = tf.Graph()
with train_graph.as_default():
    # input, output
    x = tf.placeholder(dtype=tf.float16, shape=[100, 3], name="input")
    y = tf.placeholder(dtype=tf.float16, shape=[100, 2], name="label")
    
    with tf.variable_scope('fc_1'):
      w_1 = tf.get_variable(
          'weights',
          shape=[3, 3], 
          initializer=tf.contrib.layers.xavier_initializer(), 
          dtype=tf.float16)
      b_1 = tf.get_variable(
          'bias',
          shape=[3], 
          initializer=tf.zeros_initializer(), 
          dtype=tf.float16)
      z_1 = tf.matmul(x, w_1) + b_1
    
    with tf.variable_scope('fc_2'):
      w_2 = tf.get_variable(
          'weights',
          shape=[3, 2], 
          initializer=tf.contrib.layers.xavier_initializer(), 
          dtype=tf.float16)
      b_2 = tf.get_variable(
          'bias',
          shape=[2], 
          initializer=tf.zeros_initializer(), 
          dtype=tf.float16)
      z_2 = tf.matmul(z_1, w_2) + b_2
    
    # loss functon
    cost = tf.reduce_mean(tf.square(y - z_2))
    
    optim = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)
    saver = tf.train.Saver(max_to_keep=4)
  

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        sess.run(optim, feed_dict={x:x_train, y:y_train})
        if epoch % display_step == 0:
            loss = sess.run(cost, feed_dict={x:x_train, y:y_train})
            print("epoch: %d, loss: %d" %(epoch, loss))
            # 保存训练过程中的模型
            saver.save(sess, "line_regression_model/regress.cpkt", global_step=epoch)
    print("train finished...")
    # 保存最终的模型
    saver.save(sess, "line_regression_model/regress.cpkt")
    print("final loss:", sess.run(cost, feed_dict={x:x_train, y:y_train}))
    print("weight_1:\n", sess.run(w_1))
    print("bias_1:\n", sess.run(b_1))
    print("weight_2:\n", sess.run(w_2))
    print("bias_2:\n", sess.run(b_2))

epoch: 0, loss: 0
epoch: 2, loss: 0
epoch: 4, loss: 0
epoch: 6, loss: 0
epoch: 8, loss: 0
epoch: 10, loss: 0
epoch: 12, loss: 0
epoch: 14, loss: 0
epoch: 16, loss: 0
epoch: 18, loss: 0
epoch: 20, loss: 0
epoch: 22, loss: 0
epoch: 24, loss: 0
epoch: 26, loss: 0
epoch: 28, loss: 0
train finished...
final loss: 0.396
weight_1:
 [[ 0.33544922  0.40966797  0.05038452]
 [-0.17602539  0.61669922 -0.59326172]
 [ 0.44262695 -0.27441406  0.77880859]]
bias_1:
 [-0.00751114 -0.04244995  0.05822754]
weight_2:
 [[-0.33251953  0.45068359]
 [-0.27392578  0.08459473]
 [ 0.43652344 -0.08465576]]
bias_2:
 [ 0.16186523  0.11657715]


In [43]:
tf.reset_default_graph()
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    checkpoint_path = tf.train.latest_checkpoint('line_regression_model/')
    loader = tf.train.import_meta_graph(checkpoint_path + '.meta')
    loader.restore(sess, checkpoint_path)
    w_1_new = sess.run(loaded_graph.get_tensor_by_name("fc_1/weights:0"))
    b_1_new = sess.run(loaded_graph.get_tensor_by_name("fc_1/bias:0"))
    w_2_new = sess.run(loaded_graph.get_tensor_by_name("fc_2/weights:0"))
    b_2_new = sess.run(loaded_graph.get_tensor_by_name("fc_2/bias:0"))
    
print("weight_1:\n", w_1_new)
print("bias_1:\n", b_1_new)
print("weight_2:\n", w_2_new)
print("bias_2:\n", b_2_new)

INFO:tensorflow:Restoring parameters from line_regression_model/regress.cpkt
weight_1:
 [[ 0.33544922  0.40966797  0.05038452]
 [-0.17602539  0.61669922 -0.59326172]
 [ 0.44262695 -0.27441406  0.77880859]]
bias_1:
 [-0.00751114 -0.04244995  0.05822754]
weight_2:
 [[-0.33251953  0.45068359]
 [-0.27392578  0.08459473]
 [ 0.43652344 -0.08465576]]
bias_2:
 [ 0.16186523  0.11657715]


In [44]:
import numpy as np
import tensorflow as tf

# data
x_train = np.random.random([100, 3])
y_train = np.rint(np.random.random([100, 2]))
 
epochs = 30
display_step = 2


tf.reset_default_graph()
train_graph = tf.Graph()
with train_graph.as_default():
    # input, output
    x = tf.placeholder(dtype=tf.float16, shape=[100, 3], name="input")
    y = tf.placeholder(dtype=tf.float16, shape=[100, 2], name="label")
    
    with tf.variable_scope('fc_1'):
      w_1 = tf.Variable(initial_value=w_1_new, trainable=True)
      b_1 = tf.Variable(initial_value=b_1_new, trainable=True)
      z_1 = tf.matmul(x, w_1) + b_1
    
    with tf.variable_scope('fc_2'):
      w_2 = tf.Variable(initial_value=w_2_new, trainable=True)
      b_2 = tf.Variable(initial_value=b_2_new, trainable=True)
      z_2 = tf.matmul(z_1, w_2) + b_2
      
    with tf.variable_scope('fc_3'):
      w_3 = tf.get_variable(
          'weights',
          shape=[2, 2], 
          initializer=tf.contrib.layers.xavier_initializer(), 
          dtype=tf.float16)
      b_3 = tf.get_variable(
          'bias',
          shape=[2], 
          initializer=tf.zeros_initializer(), 
          dtype=tf.float16)
      z_3 = tf.matmul(z_2, w_3) + b_3
    
    # loss functon
    cost = tf.reduce_mean(tf.square(y - z_3))
    
    optim = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)
    saver = tf.train.Saver(max_to_keep=4)
  

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    print("weight_1:\n", sess.run(w_1))
    print("bias_1:\n", sess.run(b_1))
    print("weight_2:\n", sess.run(w_2))
    print("bias_2:\n", sess.run(b_2))
    for epoch in range(epochs):
        sess.run(optim, feed_dict={x:x_train, y:y_train})
        if epoch % display_step == 0:
            loss = sess.run(cost, feed_dict={x:x_train, y:y_train})
            print("epoch: %d, loss: %d" %(epoch, loss))
            # 保存训练过程中的模型
            saver.save(sess, "line_regression_model/regress.cpkt", global_step=epoch)
    print("train finished...")
    # 保存最终的模型
    saver.save(sess, "line_regression_model/regress.cpkt")
    print("final loss:", sess.run(cost, feed_dict={x:x_train, y:y_train}))

weight_1:
 [[ 0.33544922  0.40966797  0.05038452]
 [-0.17602539  0.61669922 -0.59326172]
 [ 0.44262695 -0.27441406  0.77880859]]
bias_1:
 [-0.00751114 -0.04244995  0.05822754]
weight_2:
 [[-0.33251953  0.45068359]
 [-0.27392578  0.08459473]
 [ 0.43652344 -0.08465576]]
bias_2:
 [ 0.16186523  0.11657715]
epoch: 0, loss: 0
epoch: 2, loss: 0
epoch: 4, loss: 0
epoch: 6, loss: 0
epoch: 8, loss: 0
epoch: 10, loss: 0
epoch: 12, loss: 0
epoch: 14, loss: 0
epoch: 16, loss: 0
epoch: 18, loss: 0
epoch: 20, loss: 0
epoch: 22, loss: 0
epoch: 24, loss: 0
epoch: 26, loss: 0
epoch: 28, loss: 0
train finished...
final loss: 0.27856
